Module imports

In [28]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow import keras

Loading and training and testing the data

In [29]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info = True, as_supervised = True)

train_dataset, test_dataset = dataset['train'], dataset['test']
encoder = info.features['text'].encoder #Encoder is a reduced dimensional representaton of a set of words

print(encoder.subwords[:20])  #Seeing the first 20 words of the dictionary

['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br', 'in_', 'I_', 'that_', 'this_', 'it_', ' /><', ' />', 'was_', 'The_', 'as_']


These reviews are of different lengths and we don't have an identical length through each of the reviews and so when load elements into a matrix, we lead them by doing padding firt.

Take the longest reviews and make all the other words of same lengths by adding zeros to the end of each of those words.

In [30]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
padded_shapes = ([None],())

train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=padded_shapes)
test_dataset = test_dataset.padded_batch(BATCH_SIZE, padded_shapes=padded_shapes)

Making the model

The model is a sequencial keras model with a bi-directional layer as well as couple of dense layers

In [31]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(encoder.vocab_size,64),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                             tf.keras.layers.Dense(64,activation='relu'),
                             tf.keras.layers.Dense(1,activation='sigmoid') #probability of a review being positive or negative
                             ])  

model.compile(optimizer = tf.keras.optimizers.Adam(1e-4), loss = 'binary_crossentropy',metrics = ['accuracy'])

We are going to train and dump the history of our training object

In [32]:
history = model.fit(train_dataset,epochs=5, validation_data = test_dataset,validation_steps = 30) #epochs=5 | validation_steps = 30

Epoch 1/5
391/391 [==============================] - 798s 2s/step - loss: 0.6383 - accuracy: 0.6197 - val_loss: 0.4703 - val_accuracy: 0.8021
Epoch 2/5
391/391 [==============================] - 754s 2s/step - loss: 0.3500 - accuracy: 0.8596 - val_loss: 0.3448 - val_accuracy: 0.8625
Epoch 3/5
391/391 [==============================] - 751s 2s/step - loss: 0.2513 - accuracy: 0.9051 - val_loss: 0.3372 - val_accuracy: 0.8708
Epoch 4/5
391/391 [==============================] - 746s 2s/step - loss: 0.2122 - accuracy: 0.9250 - val_loss: 0.3796 - val_accuracy: 0.8703
Epoch 5/5
391/391 [==============================] - 756s 2s/step - loss: 0.1863 - accuracy: 0.9344 - val_loss: 0.3281 - val_accuracy: 0.8672


Padding the vectors that we sent despite their sizes

In [33]:
def pad_to_size(vec, size):
  zeros = [0]*(size-len(vec)) #padding with 0's as they don't have any menaing in our dictionary
  vec.extend(zeros)
  return vec

Generate predictions

In [34]:
def sample_predict(sentence, pad):  #We can't use model.predict like before as we have to deal with the PAD issue this time unlike before
  encoded_sample_pred_text = encoder.encode(sentence)
  if pad:
      encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text,64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32 )
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text,0))

  return predictions


Testing the created model

In [35]:
#Should return positive
sample_text = "this movie was awesome. The acting was incredible, highly recommended"
predictions = sample_predict(sample_text,pad = True)*100
print("Probability of this being a positive review",predictions, '[This is the probability of this statement being a positive review]')

#Should return positive
sample_text = "this movie was boring. The acting was mediocre, not recommended"
predictions = sample_predict(sample_text,pad = True)*100
print("Probability of this being a positive review",predictions, '[This is the probability of this statement being a positive review]')

Probability of this being a positive review [[85.081184]] [This is the probability of this statement being a positive review]
Probability of this being a positive review [[26.737755]] [This is the probability of this statement being a positive review]


#**Making more complex models**

New model

The model is a sequencial keras model with a bi-directional layer as well as couple of dense layers

In [36]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(encoder.vocab_size,64),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                             tf.keras.layers.Dense(64,activation='relu'),
                             tf.keras.layers.Dropout(0.5), #Preventing overfitting
                             tf.keras.layers.Dense(1,activation='sigmoid') #probability of a review being positive or negative
                             ])  

model.compile(optimizer = tf.keras.optimizers.Adam(1e-4), loss = 'binary_crossentropy',metrics = ['accuracy'])


We are going to train and dump the history of our training object


In [37]:
history = model.fit(train_dataset,epochs=5, validation_data = test_dataset,validation_steps = 30) #epochs=5 | validation_steps = 30

Epoch 1/5
391/391 [==============================] - 1405s 4s/step - loss: 0.6862 - accuracy: 0.5349 - val_loss: 0.6111 - val_accuracy: 0.6594
Epoch 2/5
391/391 [==============================] - 1354s 3s/step - loss: 0.4892 - accuracy: 0.7856 - val_loss: 0.3817 - val_accuracy: 0.8484
Epoch 3/5
391/391 [==============================] - 1391s 4s/step - loss: 0.3099 - accuracy: 0.8853 - val_loss: 0.3320 - val_accuracy: 0.8698
Epoch 4/5
391/391 [==============================] - 1379s 4s/step - loss: 0.2400 - accuracy: 0.9176 - val_loss: 0.3432 - val_accuracy: 0.8604
Epoch 5/5
391/391 [==============================] - 1381s 4s/step - loss: 0.2015 - accuracy: 0.9334 - val_loss: 0.3893 - val_accuracy: 0.8677


Padding the vectors that we sent despite their sizes


In [38]:
def pad_to_size(vec, size):
  zeros = [0]*(size-len(vec)) #padding with 0's as they don't have any menaing in our dictionary
  vec.extend(zeros)
  return vec



Generate predictions

In [39]:
def sample_predict(sentence, pad,model_):  #We can't use model.predict like before as we have to deal with the PAD issue this time unlike before
  encoded_sample_pred_text = encoder.encode(sentence)
  if pad:
      encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text,64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32 )
  predictions = model_.predict(tf.expand_dims(encoded_sample_pred_text,0))

  return predictions


Testing the created model

In [40]:
#Should return positive
sample_text = "this movie was awesome. The acting was incredible, highly recommended"
predictions = sample_predict(sample_text,True, model)*100
print("Probability of this being a positive review",predictions, '[This is the probability of this statement being a positive review]')

#Should return positive
sample_text = "this movie was boring. The acting was mediocre, not recommended"
predictions = sample_predict(sample_text, True, model)*100
print("Probability of this being a positive review",predictions, '[This is the probability of this statement being a positive review]')

Probability of this being a positive review [[95.9291]] [This is the probability of this statement being a positive review]
Probability of this being a positive review [[11.581442]] [This is the probability of this statement being a positive review]


#epochs=1 | validation_steps = 5

In [ ]:
49.734367 < 77.20029 #This is a good sign as the probability of a positive being positive increased 
33.938503 < 65.097885 #This is a bad sign as the probability of a negative being positive increased


#epochs=5 | validation_steps = 30

In [ ]:
85.081184 < 95.9291  #This is a good sign as the probability of a positive being positive increased 
26.737755 > 11.581442 #This is a good sign as the probability of a negative being positive decreased

#**Much detailed official documentation**

https://www.tensorflow.org/text/tutorials/text_classification_rnn
